# Setup

In [ ]:
import os
from pathlib import Path
import sys

# If we're using Google Colab, we set the environment variable to point to the relevant folder in our Google Drive:
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
    os.environ['SKIN_LESION_CLASSIFICATION'] = '/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification'

# Otherwise, we use the environment variable on our local system:
project_environment_variable = "SKIN_LESION_CLASSIFICATION"

# Path to the root directory of the project:
project_path = Path(os.environ.get(project_environment_variable))

# Relative path to /scripts (from where custom modules will be imported):
scripts_path = project_path.joinpath("scripts")

# Add this path to sys.path so that Python will look there for modules:
sys.path.append(str(scripts_path))

# Now import path_step from our custom utils module to create a dictionary to all subdirectories in our root directory:
from utils import path_setup
path = path_setup.subfolders(project_path)

Mounted at /content/drive
path['project'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification
path['images'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/images
path['notebooks'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/notebooks
path['scripts'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/scripts
path['models'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models


<a id='model_setup'></a>
# Model setup

In [ ]:
from typing import Type, Union      # For type hints
from processing import process      # Custom module for processing metadata

data_dir: Path = path["images"]     # Path to directory containing metadata.csv file
csv_filename: str = "metadata.csv"  # The filename

tvr: int = 3              # Ratio of training set to validation set. See discussion below for explanation.
seed: int = 0             # Random seed for parts of the process where randomness is called for.
keep_first: bool = False  # If False, then, for each lesion, we choose a random image to assign to our training set.
stratified: bool = True   # If True, we stratify classes so that the proportions remain as stable as possible after train/val split.
                          # If False, the proportions will be roughly similar.

to_classify: Union[list, dict] = ["mel",   # These are the lesion types we are interested in classifying.
                                  "bcc",   # Any missing ones will be grouped together as the 0-label class: no need to write "other" here.
                                  "akiec", # If 'other' is not desired, use restrict_to attribute above
                                  "nv",]   # Can also be a dictionary, like { 'malignant' : ['mel', 'bcc'], 'benign' : ['nv', 'bkl']}

train_one_img_per_lesion: Union[None, bool] = False

val_expansion_factor: Union[None,int] = 3

sample_size: Union[None, dict] = {"mel": 2000,     # Handling class imbalance by upsampling minority classes/downsampling majority classes
                                  "bcc": 2000,     # Specify how many images of each lesion diagnosis we want in our training set.
                                  "akiec": 2000,
                                  "nv": 2000,
                                  "other" : 2000,} # Could also leave out "other" here, and include e.g. "df: 2000" if we wanted to.

In [ ]:
# Create an instance of the process class with attribute values as above.
balance_ta = process(data_dir=data_dir,
                  csv_filename=csv_filename,
                  tvr=tvr,
                  seed=seed,
                  keep_first=keep_first,
                  stratified=stratified,
                  to_classify=to_classify,
                  train_one_img_per_lesion=train_one_img_per_lesion,
                  val_expansion_factor=val_expansion_factor,
                  sample_size=sample_size,)

- Loaded file '/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/images/metadata.csv'.
- Inserted 'num_images' column in dataframe, to the right of 'lesion_id' column.
- Inserted 'label' column in dataframe, to the right of 'dx' column: 
  {'bkl': 0, 'df': 0, 'vasc': 0, 'akiec': 1, 'bcc': 2, 'mel': 3, 'nv': 4}
- Added 'set' column to dataframe, with values 't1', 'v1', 'ta', and 'va', to the right of 'localization' column.
- Basic, overall dataframe (pre-train/test split): self.df
- Balancing classes in training set.
- Balanced training set (uses as many different images per lesion as possible): self.df_train
- Expanding validation set: will combine 3 predictions into one, for each lesion in val set.
- Expanded validation set (one image per lesion, repeated 3 times): self.df_val1
- Expanded validation set (use up to 3 different images per lesion, if available): self.df_val_a
- Small sample dataframes for code testing: self._df_train_code_test, self._df_val1_code_test, se

In [ ]:
import torchvision.transforms as transforms
transform = transforms.Compose([
#transforms.RandomCrop((300, 300)),
#transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
transforms.RandomRotation(180),
transforms.Resize((224,224)), # Resize images to fit ResNet input size
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])

In [ ]:
import pandas as pd
from typing import Union, List, Callable
import torchvision.models as models

source: Union[process, pd.DataFrame] = balance_ta        # Processed data to be fed into model for training.
                                                      # Must either be an instance of the process class, or a dataframe of the same format as source.df if source were an instance of the process class.
model_dir: Path = path["models"]                      # Path to directory where models/model info/model results are stored.

transform: Union[None,
                 transforms.Compose,
                 List[Callable]] = transform     # Transform to be applied to images before feeding into neural network.

filename_stem: Union[None, str] = "rn18"         # For saving model and related files. Default "rn18" (if ResNet model) or "EffNet" (if EfficientNet), or "cnn".
filename_suffix: Union[None, str] = "rot" # Something descriptive and unique for future reference. Default empty string "".

# model: Union[None, models.ResNet, models.EfficientNet] = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT) # Pre-trained model. Default: ResNet18.
model: Union[None, models.ResNet, models.EfficientNet] = models.resnet18(weights="ResNet18_Weights.DEFAULT")
unfreeze_last: Union[None, bool] = True
# overwrite: Union[None, bool] = True

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 107MB/s]


In [ ]:
# Create an instance of the resnet18 class with attribute values as above.
from multiclass_models import cnn

rn18_bal_ta_rot = cnn(source=source,
                    model_dir=model_dir,
                    transform=transform,
                    filename_stem=filename_stem,
                    filename_suffix=filename_suffix,
                    model=model,
                    unfreeze_last=unfreeze_last,)

New files will be created. 
Base filename: rn18_ta_bal_uflast_10e_rot_01
Attributes saved to file: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_ta_bal_uflast_10e_rot_01_attributes.json


In [ ]:
import time

tic = time.time()
rn18_bal_ta_rot.train()
toc = time.time()
print(f"Elapsed time: {toc - tic}")

In [ ]:
# import torch
# import torch.nn as nn

# instance = rn18_bal_ta_jit

# model = models.resnet18()
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, len(instance.label_codes))

# file_path_pth = instance.model_dir.joinpath(instance._filename + ".pth")
# state_dict = torch.load(file_path_pth)
# model.load_state_dict(state_dict)

# # model = models.efficientnet_b0()
# # num_ftrs = model.classifier[1].in_features
# # model.classifier[1] = nn.Linear(num_ftrs, len(instance.label_codes))

# instance.model = model
# instance.state_dict = state_dict

In [ ]:
instance = rn18_bal_ta_rot
instance._filename

'rn18_ta_bal_uflast_10e_rot_01'

In [ ]:
# from utils import print_header
from multiclass_models import get_probabilities
import time

instance = rn18_bal_ta_rot
instance._filename = "rn18_ta_bal_uflast_10e_rot_00"

tic = time.time()

instance.df_probabilities_val1 = get_probabilities(df=instance.df_val1,
                                                   data_dir=instance.data_dir,
                                                   model_dir=instance.model_dir,
                                                   model=instance.model,
                                                   filename=instance._filename,
                                                   label_codes=instance.label_codes,
                                                   transform=instance.transform,
                                                   batch_size=instance.batch_size,
                                                   Print=False,
                                                   save_as=instance._filename + "_val1",)

toc = time.time()

print(f"Elapsed time: {toc - tic}")

instance.df_probabilities_val_a = get_probabilities(df=instance.df_val_a,
                                                    data_dir=instance.data_dir,
                                                    model_dir=instance.model_dir,
                                                    model=instance.model,
                                                    filename=instance._filename,
                                                    label_codes=instance.label_codes,
                                                    transform=instance.transform,
                                                    batch_size=instance.batch_size,
                                                    Print=False,
                                                    save_as=instance._filename + "_val_a",)
tic = time.time()

print(f"Elapsed time: {tic - toc}")

Saving probabilities: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_ta_bal_uflast_10e_rot_00_val1_probabilities.csv
Elapsed time: 1004.2393941879272
Saving probabilities: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_ta_bal_uflast_10e_rot_00_val_a_probabilities.csv
Elapsed time: 460.6703703403473


In [ ]:
# instance = rn18_bal_ta_jit

# file_path1 = instance.model_dir.joinpath("rn18_ta_bal_uflast_10e_rndcrop_00_val1_probabilities.csv")
# file_path_a = instance.model_dir.joinpath("rn18_ta_bal_uflast_10e_rndcrop_00_val_a_probabilities.csv")

# instance.df_probabilities_val1 = pd.read_csv(file_path1, index_col=0)
# instance.df_probabilities_val_a = pd.read_csv(file_path_a, index_col=0)

In [ ]:
from utils import print_header

instance = rn18_bal_ta_rot

print_header("Validation set: one image per lesion")
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')]
display(instance.df_probabilities_val1[display_columns].head())

print_header("Validation set: all images per lesion")
display(instance.df_probabilities_val_a[display_columns].head())


VALIDATION SET: ONE IMAGE PER LESION



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv
0,HAM_0002730,ISIC_0025661,bkl,0.660780,0.122252,0.176506,0.018035,0.022427
1,HAM_0002730,ISIC_0025661,bkl,0.768884,0.033799,0.175720,0.010075,0.011522
2,HAM_0002730,ISIC_0025661,bkl,0.891202,0.023492,0.069193,0.006311,0.009801
3,HAM_0001466,ISIC_0027850,bkl,0.061633,0.000026,0.909046,0.018552,0.010743
4,HAM_0001466,ISIC_0027850,bkl,0.490021,0.000096,0.244687,0.223803,0.041394



VALIDATION SET: ALL IMAGES PER LESION



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv
0,HAM_0002730,ISIC_0026769,bkl,0.407198,0.560397,0.017732,0.000246,0.014427
1,HAM_0002730,ISIC_0026769,bkl,0.115405,0.824008,0.057889,0.000228,0.002470
2,HAM_0002730,ISIC_0025661,bkl,0.574073,0.099840,0.300622,0.014362,0.011103
3,HAM_0001466,ISIC_0031633,bkl,0.884648,0.002468,0.093574,0.015297,0.004014
4,HAM_0001466,ISIC_0027850,bkl,0.070650,0.001640,0.902797,0.013953,0.010959


In [ ]:
from collections import OrderedDict
from typing import Dict, List
from multiclass_models import final_prediction

instance = rn18_bal_ta_rot

raw_probabilities_df1: pd.DataFrame = instance.df_probabilities_val1
raw_probabilities_df_a: pd.DataFrame = instance.df_probabilities_val_a
aggregate_method: Union[None, Dict[str, List[str]]] = None#{ 'max' : ['mel', 'bcc', 'akiec'], 'min' : ['nv'], 'mean' : ['other']}
threshold_dict_help: Union[None, OrderedDict[str, float]] = None# OrderedDict([('mel',0.4), ('bcc', 0.4), ('akiec', 0.4)])
threshold_dict_hinder: Union[None, OrderedDict[str, float]] = None#OrderedDict([('nv',0.6)])
votes_to_win_dict: Union[None, OrderedDict[str, int]] = None #OrderedDict([('mel',1), ('bcc',1), ('akiec',1)])
label_codes: Dict[int, str] = instance.label_codes
prefix: Union[None, str] = 'prob_'

print_header("Validation set, one image per lesion: combining probabilities and making predictions")

instance.df_pred_val1 = final_prediction(raw_probabilities_df=raw_probabilities_df1,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')] + ['pred', 'pred_final']
display(instance.df_pred_val1[display_columns].head())

print_header("Validation set, all images per lesion: combining probabilities, making predictions, and combining predictions")

instance.df_pred_val_a = final_prediction(raw_probabilities_df=raw_probabilities_df_a,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)

display(instance.df_pred_val_a[display_columns].head())


VALIDATION SET, ONE IMAGE PER LESION: COMBINING PROBABILITIES AND MAKING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0025661,bkl,0.660780,0.122252,0.176506,0.018035,0.022427,0,0
1,HAM_0002730,ISIC_0025661,bkl,0.768884,0.033799,0.175720,0.010075,0.011522,0,0
2,HAM_0002730,ISIC_0025661,bkl,0.891202,0.023492,0.069193,0.006311,0.009801,0,0
3,HAM_0001466,ISIC_0027850,bkl,0.061633,0.000026,0.909046,0.018552,0.010743,2,2
4,HAM_0001466,ISIC_0027850,bkl,0.490021,0.000096,0.244687,0.223803,0.041394,0,2



VALIDATION SET, ALL IMAGES PER LESION: COMBINING PROBABILITIES, MAKING PREDICTIONS, AND COMBINING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0026769,bkl,0.407198,0.560397,0.017732,0.000246,0.014427,1,1
1,HAM_0002730,ISIC_0026769,bkl,0.115405,0.824008,0.057889,0.000228,0.002470,1,1
2,HAM_0002730,ISIC_0025661,bkl,0.574073,0.099840,0.300622,0.014362,0.011103,0,1
3,HAM_0001466,ISIC_0031633,bkl,0.884648,0.002468,0.093574,0.015297,0.004014,0,0
4,HAM_0001466,ISIC_0027850,bkl,0.070650,0.001640,0.902797,0.013953,0.010959,2,0


In [ ]:
# from utils import print_header
from evaluation import weighted_average_f, confusion_matrix_with_metric

instance = rn18_bal_ta_rot
map_labels = instance.label_codes

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']

txp1 = pd.crosstab(target1,prediction1,margins=True,dropna=False)
txp_a = pd.crosstab(target_a,prediction_a,margins=True,dropna=False)

beta = 2
# Weights inversely proportional to relative class size in the training set, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

instance.cm1 = confusion_matrix_with_metric(AxB=txp1,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

instance.cm_a = confusion_matrix_with_metric(AxB=txp_a,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

print_header("Confusion matrix: validation set, one image per lesion")
display(instance.cm1.fillna('_'))

print_header("Confusion matrix: validation set, all images per lesion")
display(instance.cm_a.fillna('_'))


CONFUSION MATRIX: VALIDATION SET, ONE IMAGE PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,161,4,10,17,33,225,0.715556
akiec,15,18,18,3,3,57,0.315789
bcc,11,0,68,1,2,82,0.829268
mel,33,0,0,88,33,154,0.571429
nv,62,2,10,96,"1,181","1,351",0.874167
All,282,24,106,205,"1,252","1,869",_
precision,0.570922,0.75,0.641509,0.429268,0.943291,_,0.648941



CONFUSION MATRIX: VALIDATION SET, ALL IMAGES PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,165,4,8,15,33,225,0.733333
akiec,15,17,20,2,3,57,0.298246
bcc,9,1,68,1,3,82,0.829268
mel,27,1,0,89,37,154,0.577922
nv,69,2,10,89,"1,181","1,351",0.874167
All,285,25,106,196,"1,257","1,869",_
precision,0.578947,0.68,0.641509,0.454082,0.939539,_,0.650023


In [ ]:
# from utils import print_header
from evaluation import metric_dictionary
# import pandas as pd

instance = rn18_bal_ta_rot

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']
probabilities1 = instance.df_probabilities_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']
probabilities_a = instance.df_probabilities_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

beta = 2
# Weights inversely proportional to relative class size, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

print_header("Baseline model: other metrics")

instance.metric_dict1 = metric_dictionary(target=target1,
                                          prediction=prediction1,
                                          probabilities=probabilities1)

instance.metric_dict_a = metric_dictionary(target=target_a,
                                          prediction=prediction_a,
                                          probabilities=probabilities_a)

print("\nOne image per lesion".upper())
display(pd.DataFrame(instance.metric_dict1))

print("\nAll images per lesion".upper())
display(pd.DataFrame(instance.metric_dict_a))


BASELINE MODEL: OTHER METRICS


ONE IMAGE PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.811129,0.661242,NaN,NaN,0.647099,0.640124,0.648941,0.615448,0.942383,0.944053,0.9154



ALL IMAGES PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.813269,0.662587,NaN,NaN,0.643568,0.639869,0.650023,0.618109,0.939333,0.942062,0.912609


In [ ]:
from collections import OrderedDict
from typing import Dict, List
from multiclass_models import final_prediction

instance = rn18_bal_ta_rot

raw_probabilities_df1: pd.DataFrame = instance.df_probabilities_val1
raw_probabilities_df_a: pd.DataFrame = instance.df_probabilities_val_a
aggregate_method: Union[None, Dict[str, List[str]]] = { 'max' : ['mel', 'bcc', 'akiec'], 'min' : ['nv'], 'mean' : ['other']}
threshold_dict_help: Union[None, OrderedDict[str, float]] = OrderedDict([('mel',0.4), ('bcc', 0.4), ('akiec', 0.4)])
threshold_dict_hinder: Union[None, OrderedDict[str, float]] = None#OrderedDict([('nv',0.6)])
votes_to_win_dict: Union[None, OrderedDict[str, int]] = None #OrderedDict([('mel',1), ('bcc',1), ('akiec',1)])
label_codes: Dict[int, str] = instance.label_codes
prefix: Union[None, str] = 'prob_'

print_header("Validation set, one image per lesion: combining probabilities and making predictions")

instance.df_pred_val1 = final_prediction(raw_probabilities_df=raw_probabilities_df1,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')] + ['pred', 'pred_final']
display(instance.df_pred_val1[display_columns].head())

print_header("Validation set, all images per lesion: combining probabilities, making predictions, and combining predictions")

instance.df_pred_val_a = final_prediction(raw_probabilities_df=raw_probabilities_df_a,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)

display(instance.df_pred_val_a[display_columns].head())


VALIDATION SET, ONE IMAGE PER LESION: COMBINING PROBABILITIES AND MAKING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0025661,bkl,0.660780,0.122252,0.176506,0.018035,0.022427,0,0
1,HAM_0002730,ISIC_0025661,bkl,0.768884,0.033799,0.175720,0.010075,0.011522,0,0
2,HAM_0002730,ISIC_0025661,bkl,0.891202,0.023492,0.069193,0.006311,0.009801,0,0
3,HAM_0001466,ISIC_0027850,bkl,0.061633,0.000026,0.909046,0.018552,0.010743,2,2
4,HAM_0001466,ISIC_0027850,bkl,0.490021,0.000096,0.244687,0.223803,0.041394,0,2



VALIDATION SET, ALL IMAGES PER LESION: COMBINING PROBABILITIES, MAKING PREDICTIONS, AND COMBINING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0026769,bkl,0.407198,0.560397,0.017732,0.000246,0.014427,1,1
1,HAM_0002730,ISIC_0026769,bkl,0.115405,0.824008,0.057889,0.000228,0.002470,1,1
2,HAM_0002730,ISIC_0025661,bkl,0.574073,0.099840,0.300622,0.014362,0.011103,0,1
3,HAM_0001466,ISIC_0031633,bkl,0.884648,0.002468,0.093574,0.015297,0.004014,0,0
4,HAM_0001466,ISIC_0027850,bkl,0.070650,0.001640,0.902797,0.013953,0.010959,2,0


In [ ]:
# from utils import print_header
from evaluation import weighted_average_f, confusion_matrix_with_metric

instance = rn18_bal_ta_rot
map_labels = instance.label_codes

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']

txp1 = pd.crosstab(target1,prediction1,margins=True,dropna=False)
txp_a = pd.crosstab(target_a,prediction_a,margins=True,dropna=False)

beta = 2
# Weights inversely proportional to relative class size in the training set, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

instance.cm1 = confusion_matrix_with_metric(AxB=txp1,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

instance.cm_a = confusion_matrix_with_metric(AxB=txp_a,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

print_header("Confusion matrix: validation set, one image per lesion")
display(instance.cm1.fillna('_'))

print_header("Confusion matrix: validation set, all images per lesion")
display(instance.cm_a.fillna('_'))


CONFUSION MATRIX: VALIDATION SET, ONE IMAGE PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,156,6,10,20,33,225,0.693333
akiec,14,19,18,3,3,57,0.333333
bcc,11,0,68,1,2,82,0.829268
mel,31,0,0,92,31,154,0.597403
nv,62,2,10,113,"1,164","1,351",0.861584
All,274,27,106,229,"1,233","1,869",_
precision,0.569343,0.703704,0.641509,0.401747,0.944039,_,0.648327



CONFUSION MATRIX: VALIDATION SET, ALL IMAGES PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,162,4,8,18,33,225,0.72
akiec,14,18,20,2,3,57,0.315789
bcc,9,1,68,1,3,82,0.829268
mel,27,1,0,95,31,154,0.616883
nv,69,2,9,105,"1,166","1,351",0.863064
All,281,26,105,221,"1,236","1,869",_
precision,0.576512,0.692308,0.647619,0.429864,0.943366,_,0.654185


In [ ]:
# from utils import print_header
from evaluation import metric_dictionary
# import pandas as pd

instance = rn18_bal_ta_rot

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']
probabilities1 = instance.df_probabilities_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']
probabilities_a = instance.df_probabilities_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

beta = 2
# Weights inversely proportional to relative class size, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

print_header("Baseline model: other metrics")

instance.metric_dict1 = metric_dictionary(target=target1,
                                          prediction=prediction1,
                                          probabilities=probabilities1)

instance.metric_dict_a = metric_dictionary(target=target_a,
                                          prediction=prediction_a,
                                          probabilities=probabilities_a)

print("\nOne image per lesion".upper())
display(pd.DataFrame(instance.metric_dict1))

print("\nAll images per lesion".upper())
display(pd.DataFrame(instance.metric_dict_a))


BASELINE MODEL: OTHER METRICS


ONE IMAGE PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.802033,0.662984,NaN,NaN,0.638874,0.636476,0.648327,0.603716,0.942383,0.944053,0.9154



ALL IMAGES PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.807384,0.669001,NaN,NaN,0.644097,0.641884,0.654185,0.61343,0.939333,0.942062,0.912609
